---
title: "Particle-based beam envelope tracker"
toc: false
categories:
  - space charge
  - simulation
draft: true
---

## Kapchinskij-Vladimirskij equilibrium distribution and envelope equations

The motion of intense charged particle beams is, in general, very complicated because of the Coloumb forces between particles. The distribution function $f(\mathbf{x}, \dot{\mathbf{x}}, t)$ evolves according to the Vlasov-Poisson equations 

$$
    \frac{df}{dt} 
    =
    \frac{\partial{f}}{\partial{t}} + 
    \dot{\mathbf{x} } \cdot \frac{\partial{f}}{\partial{     \mathbf{x}}} + 
    \frac{q}{m} \left( \mathbf{E} + \mathbf{v} \times \mathbf{B} \right) \cdot \frac{\partial{f}}{\partial{\dot{\mathbf{x}}}} 
    = 0,
$$ {#eq-vlasov}

where $q$ is the particle charge, $m$ is the particle mass, $\mathbf{E}$ is the electric field, and $\mathbf{B}$ is the magnetic field. $\mathbf{E}$ and $\mathbf{B}$ account for both external/applied and internal/self-generated fields. Assuming that self-generated magnetic fields are negligible and that applied fields are entirely magnetic, $\mathbf{E}(\mathbf{x}, t)$ is determined by the Poisson equation:

$$
\frac{\partial}{\partial \mathbf{x}} \mathbf{E}(\mathbf{x}, t) = \frac{1}{\epsilon_0} \int f(\mathbf{x}, 
\dot{\mathbf{x}}, t) d\dot{\mathbf{x}}.
$$ {#eq-poisson}

When the phase space is four-dimensional (4D), so that $\mathbf{x} = (x, \dot{x}, y, \dot{y})$, and when the focusing forces are time-dependent linear functions of $x$ and $y$, there is a special equilibrium solution to the Vlasov-Poisson equations known as the *Kapchinskij-Vladimiskij* (KV) distribution. The KV distribution is constructed from single-particle invariants $J_x(\mathbf{x}, \dot{\mathbf{x}})$ and $J_x(\mathbf{x}, \dot{\mathbf{x}})$, also called the *Courant-Snyder$ invariants, in the time-dependent linear system as follows:

$$
f(\mathbf{x}, \dot{\mathbf{x}}) = \delta \left( 1 - \frac{J_x(\mathbf{x}, \dot{\mathbf{x}})}{\tilde{J_x}} - \frac{J_y(\mathbf{x}, \dot{\mathbf{x}})}{\tilde{J_y}} \right)
$$ {#eq-kv}

The KV distribution is a uniformly populated ellipsoid in the 4D phase space. To prove that the distribution is in equilibrium, we have to show that it generates a linear electric field (linear in the positions $x$ and $y$), since a linear electric field preserves the single-particle invariants from which the distribution is constructed. Proving this is actually [fairly involved](https://people.frib.msu.edu/~lund/uspas/bpisc_2020/lec_set_03/ted_ho.pdf). The first step is to show that a uniform density 4D ellipsoid projects to a uniform charge density within an elliptical envelope in the $x$-$y$ plane; i.e., for a beam of line density $\lambda$ and ellipse radii $r_{x, y}$, the density is

$$
f(x, y) = 
\begin{cases}
  -\frac{\lambda}{\pi \epsilon_0 r_x r_y}, & \text{if}\ (\frac{x}{r_x})^2 + (\frac{y}{r_y})^2 < 1 \\
  0, & \text{otherwise}
\end{cases}
$$ {#eq-kv-density}

The second step is to solve the Poisson equation for this charge distribution. The end result is a simple expression for the field $E(\mathbf{x}) = (E_x, E_y)$ *inside* the ellipse:

$$
\begin{aligned}
E_x &= -\frac{\lambda}{\pi \epsilon_0} \frac{x}{(r_x + r_y) r_x} \\
E_y &= -\frac{\lambda}{\pi \epsilon_0} \frac{y}{(r_x + r_y) r_y} \\
\end{aligned}
$$ {#eq-kv-field}

The field *outside* the ellipse is nonlinear in $x$ and $y$; however, since there are no particles outside the ellipse, all particles see a linear net force from the beam and external fields.

Let's plug these fields into the single-particle equations of motion. If $\kappa(s)$ is the applied focusing strength at location $s$, then

$$
\begin{aligned}
x''(s) &= \left[ \kappa(s) - \frac{2Q}{(r_x(s) + r_y(s)) r_x(s)} \right] x(s) \\
y''(s) &= \left[ \kappa(s) - \frac{2Q}{(r_x(s) + r_y(s)) r_y(s)} \right] y(s) \\
\end{aligned}
$$ {#eq-kv-single}

Here $' = d/ds$ and $Q$ is the *perveance*, which is the charge density scaled by a factor which depends on the beam energy (higher energy = weaker space charge forces). With some additional work, we can derive equations for $r_x$ and $r_y$:

$$
\begin{aligned}
r_x''(s) &= \kappa(s) r_x(s) + \frac{\varepsilon_x^2}{r_x(s)^3} - \frac{2Q}{r_x(s) + r_y(s)} \\
r_y''(s) &= \kappa(s) r_y(s) + \frac{\varepsilon_y^2}{r_y(s)^3} - \frac{2Q}{r_x(s) + r_y(s)} \\
\end{aligned}
$$ {#eq-kv-env}

Here $\varepsilon_x = 4\sqrt{\langle xx \rangle \langle x'x' \rangle - \langle xx' \rangle \langle xx' \rangle}$ and  $\varepsilon_y = 4\sqrt{\langle yy \rangle \langle y'y' \rangle - \langle yy' \rangle \langle yy' \rangle}$ are the invariant areas of the $x$-$x'$ and $y$-$y'$ phase space ellipses; we call them the *emittances*. The first term in @eq-kv-field is the effect of linear focusing, the second term accounts the incompressibility of the phase space volume, and the third term accounts for linear space charge forces. This set of differential equations can be solved numerically to obtain the evolution of the beam *envelope*, i.e., the beam sizes $r_{x, y}$, as a function of time.

## Particle-based envelope solver

[Jeff Holmes](https://web.ornl.gov/~holmesja1/JHolmes/interests.html) had a neat idea to solve these equations using the [PyORBIT](https://github.com/PyORBIT-Collaboration/PyORBIT3) tracking code. He noted that the first term in the KV envelope equations (@eq-kv-env) resemble a single-particle equation of motion, where $r_x$ and $r_y$ are treated as the particle coordinates, and that the next two terms act as nonlinear driving forces on this fictitous particle. It's straightforward to implement these nonlinear driving forces in PyORBIT. PyORBIT represents the accelerator as *lattice* of *nodes*, where each node updates the phase space coordinates in the particle *bunch*. Quadrupole magnets are one type of node, dipole magnets are another, etc. Space-charge-driven momentum kicks are are also implemented as nodes. To solve the envelope equations, we can let the first two bunch particles $x$ and $y$ coordinates represent $r_x$ and $r_y$, and the $x'$ and $y'$ coordinates represent $r_x'$ and $r_y'$. The first term in @eq-kv-env will be applied automatically by the existing nodes in the lattice. Then we can define a new node which applies a momentum kick nonlinear terms in @eq-kv-env based on these coordinates and constants $\varepsilon_{x}$, $\varepsilon_y$, and $Q$. 

PyORBIT core algorithms are written in C++ and accessible from the Python level using wrapper functions. So to implement this node, we start by writing a C++ class.

```cpp
#include "DanilovEnvelopeSolver20.hh"

DanilovEnvelopeSolver20::DanilovEnvelopeSolver20(double perveance, double eps_x, double eps_y) : CppPyWrapper(NULL) {
  _perveance = perveance;
  _eps_x = eps_x;
  _eps_y = eps_y;
}

void DanilovEnvelopeSolver20::trackBunch(Bunch *bunch, double length) {
  // Track envelope
  double cx = bunch->x(0);
  double cy = bunch->y(0);
  double sc_term = 2.0 * _perveance / (cx + cy);
  double emit_term_x = (_eps_x * _eps_x) / (cx * cx * cx);
  double emit_term_y = (_eps_y * _eps_y) / (cy * cy * cy);
  bunch->xp(0) += length * (sc_term + emit_term_x);
  bunch->yp(0) += length * (sc_term + emit_term_y);

  // Track test particles
  double cx2 = cx * cx;
  double cy2 = cy * cy;

  double x;
  double y;
  double x2;
  double y2;

  double B;
  double C;
  double Dx;
  double Dy;
  double t1;

  double delta_xp;
  double delta_yp;
  bool in_ellipse;

  for (int i = 1; i < bunch->getSize(); i++) {
    x = bunch->x(i);
    y = bunch->y(i);

    x2 = x * x;
    y2 = y * y;

    in_ellipse = ((x2 / cx2) + (y2 / cy2)) <= 1.0;

    if (in_ellipse) {
      delta_xp = sc_term * x / cx;
      delta_yp = sc_term * y / cy;
    } else {
      // https://arxiv.org/abs/physics/0108040
      B = x2 + y2 - cx2 - cy2;
      C = x2 * cy2 + y2 * cx2 - cx2 * cy2;
      t1 = pow(0.25 * B * B + C, 0.5) + 0.5 * B;
      Dx = pow(cx2 + t1, 0.5);
      Dy = pow(cy2 + t1, 0.5);
      delta_xp = 2.0 * _perveance * x / (Dx * (Dx + Dy));
      delta_yp = 2.0 * _perveance * y / (Dy * (Dx + Dy));
    }
    bunch->xp(i) += delta_xp;
    bunch->yp(i) += delta_yp;
  }
}
```

Solver node:

In [1]:
#| code-fold: true
from orbit.lattice import AccActionsContainer
from orbit.lattice import AccLattice
from orbit.lattice import AccNode
from orbit.lattice import AccNodeBunchTracker
from orbit.utils import orbitFinalize

from orbit.ext.danilov_envelope import DanilovEnvelopeSolver20


class DanilovEnvelopeSolverNode(AccNodeBunchTracker):
    def __init__(
        self,
        solver: DanilovEnvelopeSolver20,
        name: str = None,
        kick_length: float = 0.0,
        perveance: float = 0.0,
    ) -> None:
        super().__init__(name=name)
        self.setType("DanilovEnvSolver")
        self.setLength(0.0)

        self.solver = solver
        self.kick_length = kick_length
        self.perveance = perveance
        self.active = True

    def set_active(self, setting: bool) -> None:
        self.active = setting

    def isRFGap(self) -> bool:
        # In case this node is used in linac tracking
        return False

    def trackDesign(self, params_dict):
        # In case this node is used in linac tracking
        pass

    def track(self, params_dict: dict) -> None:
        if not self.active:
            return
        bunch = params_dict["bunch"]
        self.solver.trackBunch(bunch, self.kick_length)

    def set_perveance(self, perveance: float) -> None:
        self.solver.setPerveance(perveance)

    def set_kick_length(self, kick_length: float) -> None:
        self.kick_length = kick_length


class DanilovEnvelopeSolverNode20(DanilovEnvelopeSolverNode):
    def __init__(self, eps_x: float, eps_y: float, **kwargs) -> None:
        super().__init__(solver=None, **kwargs)
        self.eps_x = eps_x
        self.eps_y = eps_y
        self.solver = DanilovEnvelopeSolver20(self.perveance, self.eps_x, self.eps_y)

    def set_emittances(self, eps_x: float, eps_y: float) -> None:
        self.eps_x = eps_x
        self.eps_y = eps_y
        self.solver.setEmittanceX(eps_x)
        self.solver.setEmittanceY(eps_y)

Lattice modification:

In [3]:
#| code-fold: true
import collections


class Parent:
    def __init__(self, node: AccNode, part_index: int, position: float, path_length: float) -> None:
        self.node = node
        self.name = self.node.getName()
        self.part_index = part_index
        self.position = position
        self.path_length = path_length


def set_max_path_length(lattice: AccLattice, length: float) -> AccLattice:
    if length:
        for node in lattice.getNodes():
            if node.getLength() > length:
                node.setnParts(1 + int(node.getLength() / length))
    return lattice


def add_danilov_envolope_solver_nodes(
    lattice: AccLattice,
    path_length_max: float,
    path_length_min: float,
    solver_node_constructor: DanilovEnvelopeSolverNode20,
    solver_node_constructor_kwargs: dict,
) -> list[DanilovEnvelopeSolverNode20]:

    nodes = lattice.getNodes()
    if not nodes:
        return

    lattice = set_max_path_length(lattice, path_length_max)

    parents = []
    length_total = running_path = rest_length = 0.0
    for node in nodes:
        for part_index in range(node.getnParts()):
            part_length = node.getLength(part_index)
            if part_length > 1.0:
                message = ""
                message += f"Warning! Node {node.getName()} has length {part_length} > 1 m. "
                message += f"Space charge algorithm may be innacurate!"
                print(message)

            parent = Parent(node, part_index, position=length_total, path_length=running_path)
            if running_path > path_length_min:
                parents.append(parent)
                running_path = 0.0

            running_path += part_length
            length_total += part_length

    if len(parents) > 0:
        rest_length = length_total - parents[-1].position
    else:
        rest_length = length_total

    parents.insert(0, Parent(node=nodes[0], part_index=0, position=0.0, path_length=rest_length))

    solver_nodes = []
    for i in range(len(parents) - 1):
        parent = parents[i]
        parent_new = parents[i + 1]

        solver_node_name = "{}:{}:".format(parent.name, parent.part_index)
        solver_node = solver_node_constructor(
            name=solver_node_name,
            kick_length=parent_new.path_length,
            **solver_node_constructor_kwargs,
        )
        parent.node.addChildNode(
            solver_node, parent.node.BODY, parent.part_index, parent.node.BEFORE
        )
        solver_nodes.append(solver_node)

    parent = parents[-1]
    solver_node = solver_node_constructor(
        name="{}:{}:".format(parent.node.getName(), parent.part_index),
        kick_length=rest_length,
        **solver_node_constructor_kwargs,
    )
    solver_nodes.append(solver_node)
    parent.node.addChildNode(solver_node, parent.node.BODY, parent.part_index, parent.node.BEFORE)

    return solver_nodes


def add_danilov_envelope_solver_nodes_20(
    lattice: AccLattice, path_length_max: float = None, path_length_min: float = 1.00e-06, **kwargs
) -> None:
    solver_nodes = add_danilov_envolope_solver_nodes(
        lattice=lattice,
        path_length_max=path_length_max,
        path_length_min=path_length_min,
        solver_node_constructor=DanilovEnvelopeSolverNode20,
        solver_node_constructor_kwargs=kwargs,
    )
    for solver_node in solver_nodes:
        name = "".join([solver_node.getName(), ":", "danilov_env_solver_20"])
        solver_node.setName(name)
    lattice.initialize()
    return solver_nodes

DanilovEnvelope20 class:

In [ ]:
import orbit.bunch_generators
import orbit.

AttributeError: module 'orbit' has no attribute 'bunch_generators'

In [ ]:
"""Envelope model for {2, 0} Danilov distribution (KV distribution)."""
import copy
import math
import time
from typing import Callable
from typing import Iterable
from typing import Self

import numpy as np

from orbit.core.bunch import Bunch

from orbit.bunch_generators import KVDist2D
from orbit.bunch_generators import GaussDist2D
from orbit.bunch_generators import WaterBagDist2D
from orbit.bunch_generators import TwissContainer
from orbit.teapot import TEAPOT_MATRIX_Lattice
from ..utils import consts

from .danilov_envelope_solver_nodes import DanilovEnvelopeSolverNode20
from .danilov_envelope_solver_lattice_modifications import add_danilov_envelope_solver_nodes_20
from .utils import calc_twiss_2d
from .utils import get_bunch_coords
from .utils import get_perveance
from .utils import get_transfer_matrix
from .utils import fit_transfer_matrix


class DanilovEnvelope20:
    """Represents envelope of {2, 0} Danilov distribution (KV distribution).

    Attributes
    ----------
    params : ndarray, shape(4,)
        The envelope parameters [cx, cx', cy, cy']. The cx and cy parameters
        represent the envelope extent along the x and y axis; cx' and cy' are
        their derivatives with respect to the distance x.
    eps_x : float
        The rms emittance of the x-x' distribution: sqrt(<xx><x'x'> - <xx'><xx'>).
    eps_y : float
        The rms emittance of the y-y' distribution: sqrt(<yy><y'y'> - <yy'><yy'>).
    mass : float
        Particle [GeV/c^2].
    kin_energy : float
        Particle kinetic energy [GeV].
    intensity : float
        Bunch intensity (number of particles).
    length : float
        Bunch length [m].
    perveance : float
        Dimensionless beam perveance.
    """

    def __init__(
        self,
        eps_x: float,
        eps_y: float,
        mass: float,
        kin_energy: float,
        length: float,
        intensity: int,
        params: Iterable[float] = None,
    ) -> None:
        self.eps_x = eps_x
        self.eps_y = eps_y
        self.mass = mass
        self.kin_energy = kin_energy

        self.length = length
        self.line_density = None
        self.perveance = None
        self.set_intensity(intensity)

        self.params = params
        if self.params is None:
            cx = 2.0 * np.sqrt(self.eps_x * 4.0)
            cy = 2.0 * np.sqrt(self.eps_y * 4.0)
            self.params = [cx, 0.0, cy, 0.0]
        self.params = np.array(self.params)

    def set_intensity(self, intensity: int) -> None:
        self.intensity = intensity
        self.line_density = intensity / self.length
        self.perveance = get_perveance(self.mass, self.kin_energy, self.line_density)

    def set_length(self, length: float) -> None:
        self.length = length
        self.set_intensity(self.intensity)

    def set_params(self, params: np.ndarray) -> None:
        self.params = np.copy(params)

    def copy(self) -> Self:
        return copy.deepcopy(self)

    def cov(self) -> np.ndarray:
        """Return covariance matrix.

        See Table II here: https://journals.aps.org/prab/abstract/10.1103/PhysRevSTAB.7.024801
        Note typo for <x'x'>: first term should be rx'^2, not rx^2.
        """
        (cx, cxp, cy, cyp) = self.params
        cov_matrix = np.zeros((4, 4))
        cov_matrix[0, 0] = 0.25 * cx**2
        cov_matrix[2, 2] = 0.25 * cy**2
        cov_matrix[1, 1] = 0.25 * cxp**2 + 4.0 * (self.eps_x / cx) ** 2
        cov_matrix[3, 3] = 0.25 * cyp**2 + 4.0 * (self.eps_y / cy) ** 2
        cov_matrix[0, 1] = cov_matrix[1, 0] = 0.25 * cx * cxp
        cov_matrix[2, 3] = cov_matrix[3, 2] = 0.25 * cy * cyp
        return cov_matrix

    def set_cov(self, cov_matrix: np.ndarray) -> None:
        """Set envelope parameters from covariance matrix."""
        self.eps_x = np.sqrt(np.linalg.det(cov_matrix[0:2, 0:2]))
        self.eps_y = np.sqrt(np.linalg.det(cov_matrix[2:4, 2:4]))
        cx = np.sqrt(4.0 * cov_matrix[0, 0])
        cy = np.sqrt(4.0 * cov_matrix[2, 2])
        cxp = 2.0 * cov_matrix[0, 1] / np.sqrt(cov_matrix[0, 0])
        cyp = 2.0 * cov_matrix[2, 3] / np.sqrt(cov_matrix[2, 2])
        params = np.array([cx, cxp, cy, cyp])
        self.set_params(params)

    def rms(self) -> np.ndarray:
        return np.sqrt(np.diagonal(self.cov()))

    def twiss(self) -> dict[str, float]:
        """Return (alpha_x, beta_x, alpha_y, beta_y)."""
        cov_matrix = self.cov()
        alpha_x, beta_x, emittance_x = calc_twiss_2d(cov_matrix[0:2, 0:2])
        alpha_y, beta_y, emittance_y = calc_twiss_2d(cov_matrix[2:4, 2:4])

        results = {}
        results["alpha_x"] = alpha_x
        results["alpha_y"] = alpha_y
        results["beta_x"] = beta_x
        results["beta_y"] = beta_y
        results["emittance_x"] = emittance_x
        results["emittance_y"] = emittance_y
        return results

    def set_twiss(
        self,
        alpha_x: float = None,
        beta_x: float = None,
        alpha_y: float = None,
        beta_y: float = None,
    ) -> None:
        twiss_params = self.twiss()
        if alpha_x is None:
            alpha_x = twiss_params["alpha_x"]
        if alpha_y is None:
            alpha_y = twiss_params["alpha_y"]
        if beta_x is None:
            beta_x = twiss_params["beta_x"]
        if beta_y is None:
            beta_y = twiss_params["beta_y"]

        gamma_x = (1.0 + alpha_x**2) / beta_x
        gamma_y = (1.0 + alpha_y**2) / beta_y
        cov_matrix = np.zeros((4, 4))
        cov_matrix[0, 0] = beta_x * self.eps_x
        cov_matrix[2, 2] = beta_y * self.eps_y
        cov_matrix[1, 1] = gamma_x * self.eps_x
        cov_matrix[3, 3] = gamma_y * self.eps_y
        cov_matrix[0, 1] = cov_matrix[1, 0] = -alpha_x * self.eps_x
        cov_matrix[2, 3] = cov_matrix[3, 2] = -alpha_y * self.eps_y
        self.set_cov(cov_matrix)

    def sample(self, size: int, dist: str = "kv") -> np.ndarray:
        twiss_params = self.twiss()
        twiss_x = TwissContainer(
            twiss_params["alpha_x"],
            twiss_params["beta_x"],
            twiss_params["emittance_x"],
        )
        twiss_y = TwissContainer(
            twiss_params["alpha_y"],
            twiss_params["beta_y"],
            twiss_params["emittance_y"],
        )

        if dist == "kv":
            dist = KVDist2D(twiss_x, twiss_y)
        elif dist == "gaussian":
            dist = GaussDist2D(twiss_x, twiss_y)
        elif dist == "waterbag":
            dist = WaterBagDist2D(twiss_x, twiss_y)
        else:
            raise ValueError

        samples = np.zeros((size, 6))
        for i in range(size):
            (x, xp, y, yp) = dist.getCoordinates()
            z = np.random.uniform(-0.5 * self.length, 0.5 * self.length)
            samples[i, :] = [x, xp, y, yp, z, 0.0]
        return samples

    def from_bunch(self, bunch: Bunch) -> np.ndarray:
        """Set envelope parameters from Bunch."""
        self.params = np.zeros(4)
        self.params[0] = bunch.x(0)
        self.params[1] = bunch.xp(0)
        self.params[2] = bunch.y(0)
        self.params[3] = bunch.yp(0)
        return self.params

    def to_bunch(self, size: int = 0, env: bool = True) -> Bunch:
        """Create Bunch object from envelope parameters.

        Parameters
        ----------
        size : int
            Number of macroparticles in the bunch. These are the number of "test"
            particles not counting the first particle, which stores the envelope
            parameters.
        env : bool
            If False, do not store the envelope parameters as the first particle.

        Returns
        -------
        Bunch
        """
        bunch = Bunch()
        bunch.mass(self.mass)
        bunch.getSyncParticle().kinEnergy(self.kin_energy)

        if env:
            (cx, cxp, cy, cyp) = self.params
            bunch.addParticle(cx, cxp, cy, cyp, 0.0, 0.0)

        if size:
            samples = self.sample(size)
            for i in range(size):
                bunch.addParticle(*samples[i])

            macrosize = self.intensity / size
            if self.intensity == 0.0:
                macrosize = 1.0
            bunch.macroSize(macrosize)

        return bunch


class DanilovEnvelopeMonitor20:
    def __init__(self, verbose: int = 0) -> None:
        self.verbose = verbose
        self.distance = 0.0
        self._pos_old = 0.0
        self._pos_new = 0.0

        self.history = {}
        for key in [
            "s",
            "xrms",
            "yrms",
        ]:
            self.history[key] = []

    def package(self) -> None:
        history = copy.deepcopy(self.history)
        for key in history:
            history[key] = np.array(history[key])
        history["s"] -= history["s"][0]
        return history

    def __call__(self, params_dict: dict) -> None:
        bunch = params_dict["bunch"]
        node = params_dict["node"]

        self._pos_new = params_dict["path_length"]
        if self._pos_old > self._pos_new:
            self._pos_old = 0.0
        self.distance += self._pos_new - self._pos_old
        self._pos_old = self._pos_new

        x_rms = bunch.x(0) * 0.5
        y_rms = bunch.y(0) * 0.5

        self.history["s"].append(self.distance)
        self.history["xrms"].append(x_rms)
        self.history["yrms"].append(y_rms)

        if self.verbose:
            print("s={:0.3f} x_rms={:0.2f}, y_rms={:0.2f}".format(self.distance, x_rms, y_rms))


class DanilovEnvelopeTracker20:
    def __init__(self, lattice: AccLattice, path_length_max: float = None) -> None:
        self.lattice = lattice
        self.solver_nodes = self.add_solver_nodes(
            path_length_min=1.00e-06, path_length_max=path_length_max
        )

        # Lower bounds on envelope parameters
        self.lb = np.zeros(4)
        self.lb[0] = +1.00 - 12
        self.lb[1] = -np.inf
        self.lb[2] = +1.00e-12
        self.lb[3] = -np.inf

        # Upper bounds on envelope parameters
        self.ub = np.zeros(4)
        self.ub[0] = np.inf
        self.ub[1] = np.inf
        self.ub[2] = np.inf
        self.ub[3] = np.inf

    def add_solver_nodes(
        self,
        path_length_min: float,
        path_length_max: float,
    ) -> list[DanilovEnvelopeSolverNode20]:

        self.solver_nodes = add_danilov_envelope_solver_nodes_20(
            lattice=self.lattice,
            path_length_min=path_length_min,
            path_length_max=path_length_max,
            perveance=0.0,  # will update based on envelope
            eps_x=1.0,  # will update based on envelope
            eps_y=1.0,  # will update based on envelope
        )
        return self.solver_nodes

    def toggle_solver_nodes(self, setting: bool) -> None:
        for node in self.solver_nodes:
            node.active = setting

    def update_solver_node_parameters(self, envelope: DanilovEnvelope20) -> None:
        for solver_node in self.solver_nodes:
            solver_node.set_perveance(envelope.perveance)
            solver_node.set_emittances(envelope.eps_x * 4.0, envelope.eps_y * 4.0)

    def track(
        self, envelope: DanilovEnvelope20, periods: int = 1, history: bool = False
    ) -> None | dict[str, np.ndarray]:
        self.update_solver_node_parameters(envelope)

        monitor = DanilovEnvelopeMonitor20()
        action_container = AccActionsContainer()
        if history:
            action_container.addAction(monitor, AccActionsContainer.ENTRANCE)
            action_container.addAction(monitor, AccActionsContainer.EXIT)

        bunch = envelope.to_bunch()
        for period in range(periods):
            self.lattice.trackBunch(bunch, actionContainer=action_container)

        envelope.from_bunch(bunch)

        if history:
            return monitor.package()

    def transfer_matrix(self, envelope: DanilovEnvelope20) -> np.ndarray:
        bunch = envelope.to_bunch(size=0, env=True)

        if envelope.perveance == 0:
            self.toggle_solver_nodes(False)
            matrix = get_transfer_matrix(self.lattice, bunch)
            self.toggle_solver_nodes(True)
            return matrix

        step_arr = np.ones(6) * 1.00e-06
        step_reduce = 20.0

        bunch.addParticle(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        bunch.addParticle(step_arr[0] / step_reduce, 0.0, 0.0, 0.0, 0.0, 0.0)
        bunch.addParticle(0.0, step_arr[1] / step_reduce, 0.0, 0.0, 0.0, 0.0)
        bunch.addParticle(0.0, 0.0, step_arr[2] / step_reduce, 0.0, 0.0, 0.0)
        bunch.addParticle(0.0, 0.0, 0.0, step_arr[3] / step_reduce, 0.0, 0.0)
        bunch.addParticle(step_arr[0], 0.0, 0.0, 0.0, 0.0, 0.0)
        bunch.addParticle(0.0, step_arr[1], 0.0, 0.0, 0.0, 0.0)
        bunch.addParticle(0.0, 0.0, step_arr[2], 0.0, 0.0, 0.0)
        bunch.addParticle(0.0, 0.0, 0.0, step_arr[3], 0.0, 0.0)

        self.lattice.trackBunch(bunch)

        X = get_bunch_coords(bunch)
        X = X[:, (0, 1, 2, 3)]
        X = X[1:, :]  # ignore first particle, which tracks envelope parameters

        M = np.zeros((4, 4))
        for i in range(4):
            for j in range(4):
                x1 = step_arr[i] / step_reduce
                x2 = step_arr[i]
                y0 = X[0, j]
                y1 = X[i + 1, j]
                y2 = X[i + 1 + 4, j]
                M[j, i] = ((y1 - y0) * x2 * x2 - (y2 - y0) * x1 * x1) / (x1 * x2 * (x2 - x1))
        return M

    def match_zero_sc(self, envelope: DanilovEnvelope20) -> None:
        self.toggle_solver_nodes(False)
        bunch = envelope.to_bunch(size=0, env=False)
        matrix_lattice = TEAPOT_MATRIX_Lattice(self.lattice, bunch)
        lattice_params = matrix_lattice.getRingParametersDict()
        self.toggle_solver_nodes(True)

        alpha_x = lattice_params["alpha x"]
        alpha_y = lattice_params["alpha y"]
        beta_x = lattice_params["beta x [m]"]
        beta_y = lattice_params["beta y [m]"]
        envelope.set_twiss(alpha_x, beta_x, alpha_y, beta_y)

    def match(
        self, envelope: DanilovEnvelope20, periods: int = 1, method: str = "least_squares", **kwargs
    ) -> None:
        if envelope.perveance == 0.0:
            return self.match_zero_sc(envelope)

        def loss_function(params: np.ndarray) -> np.ndarray:
            envelope.set_params(params)

            loss = 0.0
            for period in range(periods):
                self.track(envelope)
                residuals = envelope.params - params
                residuals = 1000.0 * residuals
                loss += np.mean(np.abs(residuals))
            return loss / float(periods)

        if method == "least_squares":
            kwargs.setdefault("xtol", 1.00e-12)
            kwargs.setdefault("ftol", 1.00e-12)
            kwargs.setdefault("gtol", 1.00e-12)
            kwargs.setdefault("verbose", 2)

            result = scipy.optimize.least_squares(
                loss_function, envelope.params.copy(), bounds=(self.lb, self.ub), **kwargs
            )
            return result
        elif method == "minimize":
            result = scipy.optimize.minimize(
                loss_function,
                envelope.params.copy(),
                bounds=scipy.optimize.Bounds(self.lb, self.ub),
                **kwargs,
            )
        else:
            raise ValueError